#### Data pre-processing
- run script to generate csv files that can be uploaded to MySQL via an sql script

#### 0. unpacking data files
- script to preview dataset to choose relevant columns to save in unpacking <br>
- script to unpack full dataset but saving only the relevant columns <br>

In [ ]:
# import libraries
import gzip
import json
import pandas as pd
import time

In [ ]:
# preview first n lines
# files to unpack: Books_5.json.gz, meta_books.json.gz, goodreads_reviews_dedup.json.gz, 

data, n = [], 100 # n = number of lines to extract
with gzip.open('goodreads_reviews_dedup.json.gz') as f:
    for i, line in enumerate(f):
        if i >= n:
            break  # Stop after reading the first 100 lines
        else:
            # Parse the line as a JSON object
            json_object = json.loads(line)
            
            # Extract only the first 100 key-value pairs (can change)
            limited_json_object = dict(list(json_object.items())[:100])
            data.append(limited_json_object)
            
# convert list to df
df = pd.DataFrame.from_dict(data)
# preview df
df

In [ ]:
# # list columns; if need to copy-paste
# df.columns

In [ ]:
## function to unpack json.gz to csv

def extract_to_csv(json_gz_file, csv_filename, cols_to_extract):
    '''
    Extracts data from a gzipped JSON file and saves the data to a CSV file.

    Parameters:
    json_gz_file (str): The path to the input gzipped JSON file.
    csv_filename (str): The desired name of the output CSV file.
    cols_to_extract (list): A list of column names to be extracted from the JSON file.

    Returns:
    None. Saves the data to a CSV file.

    '''
    # Initialize an empty list to store the data
    data_list = []

    # Initialize a counter for progress tracking
    line_counter = 0

    # Record the start time
    start_time = time.time()

    # Read the gzip file
    with gzip.open(json_gz_file, 'r') as f:
        # Read line by line and extract desired columns
        for line in f:
            line_counter += 1
            if line_counter % 1000000 == 0:
                current_time = time.time()
                elapsed_time = int(current_time - start_time)
                print(f"Processed {line_counter} lines in {elapsed_time} seconds.")
            data = json.loads(line.decode('utf-8'))
            extracted_data = {column: data.get(column) for column in cols_to_extract}
            data_list.append(extracted_data)

    # Create a DataFrame from the extracted data
    df = pd.DataFrame(data_list)

    # Save the DataFrame to a CSV file
    df.to_csv(csv_filename, index=False)

    # Display the DataFrame
    df.head()

#### 1 Pre-processing data
- Amazon dataset: preprocessing to supplement author information, generating individual relations tables in ER
- goodreads: preprocessing to the form RDMS schema require
- blackwells (AI-generated reviews): supplementing with asin, as well as randomly generated ratings and n-votes. Code to artificially generate reviews are in a separate notebook.

##### 1.1 Preprocessing Amazon data

In [ ]:
# generate 'user', 'review', 'user_review', 'item_review' tables

amazon5coredf = pd.read_csv('Books_5.csv')

# 'user' relation
userdf= amazon5coredf[['reviewerID','reviewerName','verified']]
user_df_duplicates_removed = userdf.drop_duplicates(subset=['reviewerID'], keep='first')
user_df_duplicates_removed.to_csv('user.csv', index=False)

# 'review' relation
reviewdf= amazon5coredf.reset_index().rename(columns={'index': 'rid'})
reviewdf['rid']+=1
review_df= reviewdf[['rid','overall','reviewTime','summary','unixReviewTime','vote']]
review_df.to_csv('review.csv', index=False) 

# 'user_review' relation
user_reviewdf = reviewdf[['rid','reviewerID']]
user_reviewdf.to_csv('user_review.csv', index=False) 

# item_review relation
item_reviewdf = reviewdf[['rid','asin']].rename(columns={'asin': 'book_asin'})
item_reviewdf.to_csv('item_review.csv', index=False)


In [ ]:
##For AMAZON FULL BOOK INFO

amazonmeta = []
with gzip.open('meta_books.json.gz') as f:
    for i, line in enumerate(f):
#         if i >= 10000:
#             break  # Stop after reading the first 1000000 lines
#         else:
#             # Parse the line as a JSON object
            json_object = json.loads(line)
            
            # Extract only the first 6 key-value pairs
            limited_json_object = dict(list(json_object.items())[:100])
            amazonmeta.append(limited_json_object)

df= pd.DataFrame.from_dict(amazonmeta)
df3 = df.fillna('')
df4 = df3[df3.title.str.contains('getTime')] # unformatted rows
amazonmetadf= df3[~df3.title.str.contains('getTime')] # filter those unformatted rows

In [ ]:
##For AUTHOR INFO TO CREATE AUTHOR TABLE

authordata = []
with gzip.open('goodreads_book_authors.json.gz') as f:
    for i, line in enumerate(f):
#         if i >= 10:
#             break  # Stop after reading the first 100 lines
#         else:
#             # Parse the line as a JSON object
            json_object = json.loads(line)
            
            # Extract only the first 6 key-value pairs
            limited_json_object = dict(list(json_object.items())[:100])
            authordata.append(limited_json_object)

authordf= pd.DataFrame.from_dict(authordata)

In [ ]:
##For AUTHOR INFO TO AUGMENT AMAZON DATASET

kagglebookdf = pd.read_csv('books_data.csv')

##Convert authors column from string into array

def safe_literal_eval(value):
    if not pd.isna(value):
        return ast.literal_eval(value)
    return value  # Keep NaN values as they are

kagglebookdf['authors'] = kagglebookdf['authors'].apply(safe_literal_eval)

In [ ]:
##For BOOK INFO TO AUGMENT AMAZON FULL BOOK INFO

goodreads_bookdf = pd.read_csv('goodreads_books.csv')

In [ ]:
# Supplement Authors in main dataset

####Replace empty cells with NaN

def replace_empty_array(cell):
    if isinstance(cell, list) and not cell:
        return np.nan
    return cell

###Convert cells to array 

def convert_to_array(cell):
    if cell is not np.nan:
        return [cell]
    return np.nan

amazonmetadf = amazonmetadf.applymap(replace_empty_array)

overalldf = amazonmetadf.replace('', np.nan)


###Join Author table in Goodreads onto Amazon Table and convert author list to array

merged_df = pd.merge(overalldf, authordf, left_on='brand', right_on='name', how='left')
merged_df['author'] = merged_df['name']
merged_df = merged_df.drop(['brand','name','author_id','average_rating','text_reviews_count','ratings_count'], axis=1)
merged_df['author'] = merged_df['author'].apply(convert_to_array)


###Join Author table in Kaggle onto Amazon table to create a complete set of authors (where possible) in Amazon set

author_dict = kagglebookdf.set_index('Title')['authors'].to_dict()

def update_author(row):
    title = row['title']
    authors = author_dict.get(title)
    
    if authors is not None:
        if row['author'] is np.nan:
            row['author'] = authors if isinstance(authors, list) else [authors]
        else:
            if not isinstance(authors, float):
                if not isinstance(row['author'], list):
                    row['author'] = [row['author']]
                for author in authors:
                    if author not in row['author']:
                        row['author'].append(author)
    return row

merged_df = merged_df.apply(update_author, axis=1)

merged_df= pd.merge(merged_df, goodreads_bookdf, left_on='asin', right_on='isbn', how='left')


In [ ]:
## clean main dataset (price, rank)

# Data Cleaning of Main Merged Df for Items Relation

#Convert price to float for price column
def clean_and_convert_price(price_str):
    try:
        if isinstance(price_str, str):
            cleaned_price = price_str.replace('$', '').replace(',', '')
            return float(cleaned_price)
        else:
            return price_str
    except ValueError:
        return np.nan
    
merged_df['price'] = merged_df['price'].apply(clean_and_convert_price)

# Convert rank to int for rank column
def extract_integer(text):
    try:
        match = re.search(r'(\d[\d,]*)', text)
        if match:
            return int(match.group().replace(',', ''))
        else:
            return np.nan
    except (TypeError, ValueError,AttributeError):
        return np.nan

# Apply the function to the 'Rank' column
merged_df['rank'] = merged_df['rank'].apply(extract_integer)

# # Display the modified DataFrame
# merged_df= merged_df.rename(columns={'description_x': 'description','title_x':'title'})

In [ ]:
# Break up tables for export

# 'item' relation
itemdf = merged_df[['asin','title','description','price','rank','publisher','publication_year']]
itemdf.to_csv('items.csv', index=False)

# 'item_author' relation
author_expanded_df = merged_df.explode('author')
author_expanded_df = pd.merge(author_expanded_df, authordf, left_on='author', right_on='name', how='left')
item_author = author_expanded_df[['asin','author_id']].dropna()
item_author.reset_index().drop(columns=['index'])
item_author.to_csv('item_author.csv', index=False)

# 'item_category' and 'category' relations

cat_expanded_df = merged_df.explode('category')
item_cat = cat_expanded_df[['asin','category']].dropna()

item_cat2 = item_cat.reset_index().drop(columns=['index'])
item_cat3 = item_cat2[~(item_cat2['category']=='Books')]

unique_categories = item_cat3['category'].unique()
category_dict = {index+1: category for index, category in enumerate(unique_categories)}
categorydf= pd.Series(category_dict).to_frame().reset_index()
categorydf = categorydf.rename(columns={'index': 'cid', 0:'name'})

item_categorydf = pd.merge(item_cat3, categorydf, left_on='category', right_on='name', how='left').drop(columns=['category','name'])

categorydf.to_csv('category.csv', index=False) 
item_categorydf.to_csv('item_category.csv', index=False) 

# 'also_bought' and 'also_viewed' relations 

bought_expanded_df = merged_df.explode('also_buy')
also_bought = bought_expanded_df[['asin','also_buy']].dropna()

viewed_expanded_df = merged_df.explode('also_view')
also_viewed = viewed_expanded_df[['asin','also_view']].dropna()

#Export
also_viewed.to_csv('also_viewed.csv', index=False) 
also_bought.to_csv('also_bought.csv', index=False) 

##### 1.2 Preprocessing Goodreads data

In [ ]:
# Load datasets
df_goodreads_reviews = pd.read_csv('goodreads_reviews.csv')
df_goodreads_books = pd.read_csv('goodreads_books.csv')
df_asin_amazon = pd.read_csv('asin_amazon.csv')

# extract required columns
df_goodreads_reviews = df_goodreads_reviews[['book_id', 'rating', 'review_text', 'n_votes']] 
df_goodreads_books = df_goodreads_books[['isbn', 'book_id']]

# incorporate 'isbn' to reviews table
df_external = pd.merge(df_goodreads_reviews, df_goodreads_books, 
                       left_on = 'book_id', right_on = 'book_id', how = 'left')

# remove NaN values, rename columns to be consistent with ER
df_external_dropped = df_external.dropna()
df_external_dropped = df_external_dropped.rename(columns = {'isbn': 'asin'})
df_external_dropped['platform'] = 'goodreads'
df_external_dropped = df_external_dropped[['rating', 'review_text', 'n_votes', 'platform', 'asin']]
# select a random subset of data to lower data size; 
    # only using this as a proof-of-concept demonstrate feasibility of merge an external dataset to main (Amazon) dataset
df_external_select = df_external_dropped.sample(frac=0.37, random_state = 73)
df_external_select

#####  1.3 Preprocessing Artificially generated reviews (aka 'blackwells'), and merging with goodreads data

In [ ]:
df = pd.read_csv('10k_gen_text_review.csv')
df = df.rename(columns={'Title': 'title'})

# load metabooks data to join on 'title' to get 'asin'
df_books = pd.read_csv('meta_books.csv')
df_title_asin = df_books[['title', 'asin']]

In [ ]:
# define a wrapper function to perform 'v-lookup'
def filterWrapper(fulldataframe, HeaderName="", lst=[""]):
    df_copy = fulldataframe.copy()
    if type(lst[0]) is str:
        lst_lowercase = [i.lower() for i in lst]
        df_out = df_copy[df_copy[HeaderName].str.lower().isin(lst_lowercase)]
    elif type(lst[0]) is int:
        df_out = df_copy[df_copy[HeaderName].isin(lst)]
    return df_out

In [ ]:
# wrapper to perform 'v-lookup'
df_out = filterWrapper(df_title_asin, 'title', df['title'].to_list())
df_out['title'] = df_out['title'].str.lower() # df_out has columns 'title' and 'asin'

# incorporate 'asin' to df from blackwells
merged_df = df.merge(df_out, on='title', how='left')
merged_df_dropped = merged_df.dropna()

# rename columns
df_gen = merged_df.rename(columns={'GeneratedReview': 'review_text'})
df_gen = df_gen[['review_text', 'asin']]

In [ ]:
# incorporate randomly generated ratings and rating votes
import numpy as np

# Set the seed for reproducibility
np.random.seed(0)

# Add the 'rating' column with random integer values from 1 to 5
df_gen['rating'] = np.random.randint(1, 6, size=len(df_gen))

# Add the 'n_votes' column with Poisson distributed integer values from 0 to 100, lambda=37
df_gen['n_votes'] = np.random.poisson(lam=37, size=len(df_gen))

# Add the 'platform' column with the value 'blackwells'
df_gen['platform'] = 'blackwells'

# save to csv
df_gen.to_csv('blackwells.csv', index=False)

In [ ]:
# merging with goodreads
df_blackwells = pd.read_csv('blackwells.csv')
df_external_combined = pd.concat([df_external_select, df_blackwells], axis=0).reset_index(drop=True)

# add primary key 'eid'
df_external_combined = df_external_combined.reset_index().rename(columns={'index':'eid'})
df_external_combined['eid'] += 1

# split external into 'item_external' and 'external' relations
df_external = df_external_combined[['eid', 'review_text', 'rating', 'n_votes', 'platform']]
df_item_external = df_external_combined[['eid', 'asin']]

# save to csv
df_external.to_csv('external.csv', index = False)
df_item_external.to_csv('item_external.csv', index = False)